# 実験を開始する

これは、実験パッケージを用意して実験を行うためのノートブックです。  

## ◆◆◆開発メモ◆◆◆

- 検証中
- launch binderのリンク作成が出来ていない

## 1. 実験用の空リポジトリを作成する
実験リポジトリを作成する準備として、[リポジトリの作成方法](https://gin.g-node.org/G-Node/Info/wiki/Web+Interface#creating-a-repository)を参考にデータガバナンス機能で実験用の空リポジトリを作成してください。

1. [データガバナンス機能に遷移する](http://dg02.dg.rcos.nii.ac.jp/)

## 2. ステップ1で作成した空リポジトリに実験パッケージをコピーする

1. 以下のセルを実行すると入力フォームが現れます。ステップ1で作成した空リポジトリの名前とSSH URLを入力してください。

In [ ]:
repository_title = input()

In [ ]:
ssh_url = input()

2. 空リポジトリ名のフォルダを作成し、実験パッケージをコピーします。

In [ ]:
!mkdir ~/.tmp/
!git init ~/.tmp/$repository_title
!cp -r ~/WORKFLOW/PACKAGE/* ~/.tmp/$repository_title/
!git commit -m '1st commit'
!git remote add $ssh_url
!git push

## 3. 上記リポジトリをサブデータセットとして設定する

In [ ]:
!git submodule add $ssh_url ~/experiments/repository_title

## 4. pipeline.jsonに実験名を追記する

In [ ]:
import json

with open('/home/jovyan/experiments/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(repository_title)

with open('/home/jovyan/experiments/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)

## 5. 変更内容を書き戻す

1. 書き戻しメッセージを変更したい場合は以下のセルでsave_messageに代入する値を編集してください。

In [ ]:
save_message = '実験の作成'

2. 変更内容を書き戻す

In [ ]:
import papermill as pm

pm.execute_notebook(
    '../util/datalad_save_push.ipynb',
    '-',
    parameters = dict(SAVE_MESSAGE = save_message)
)

## 6. 複製したパッケージのコンテナを立ち上げて、ソースコードのJupyterNotebookに遷移する

※下のリンクは未完成

[![Binder](https://binder.cs.rcos.nii.ac.jp/badge_logo.svg)](https://binder.cs.rcos.nii.ac.jp/v2/gh/[ユーザー名]/['/home/jovyan/[project-title]/[wxperiment-title]']/main)